#### 1. **Model Runner Step Sync And Async Testing**

Notebook tests `ModelRunnerStep` within a serving graph to enable real-time monitoring and drift detection with MLRun. 
The main focus is to test te preformance of sync and async usage.

In [1]:
# Import mlrun and create project instance
import mlrun
mlrun.set_env_from_file('../../cust_cs.env')

image = "mlrun/mlrun"
project_name = "monitored-model-runner-naive"
project = mlrun.get_or_create_project(project_name, context="./",user_project=True, allow_cross_project=True)

> 2025-11-02 13:10:12,294 [info] Project loaded successfully: {"project_name":"monitored-model-runner-naive-matanz"}


In [2]:
# Import tools
import pandas as pd
from sklearn.svm import SVC
import pickle
from sklearn.datasets import load_iris
from mlrun.features import Feature

In [3]:
# Train simple SVM model on Iris dataset, save it and reformat the DS as list
iris = load_iris()
clf = SVC()           
clf.fit(iris.data, iris.target)
with open("SVM.pkl", "wb") as fh:
    pickle.dump(clf, fh)
iris_data = iris["data"].tolist()

# load the dataset again as a DF
iris = load_iris()
train_set = pd.DataFrame(
    iris["data"],
    columns=["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm"],
)

# Create a Model Artifact in the project using the trained model
model_name = "SVM"
model_artifact = project.log_model(
    model_name,
    model_file="SVM.pkl",
    training_set=train_set,
    framework="sklearn",
    outputs=[Feature(name="label")],
)

#### 5. Define your function and ModelRunnerStep

Define functions to all the edge cases

In [4]:
# Config the code path and the serving function sync and async
bouth_code_path = r"model_class_bouth.py"

function = project.set_function(func=bouth_code_path,image="mlrun/mlrun",kind="serving",name="bouth")

In [5]:
from mlrun.serving.states import ModelRunnerStep

model_runner_step = ModelRunnerStep(
    name="my_runner", model_selector="MyModelSelector",model_selector_parameters={"name":"my-selector"})

for i in range(100):
    model_runner_step.add_model(
        model_class="MyModel",
        endpoint_name=f"my-{i}-model",
        model_artifact=model_artifact,
        input_path="inputs.here",
        result_path="outputs",
        outputs=["label"],
        execution_mechanism="shared_executor",
    )

#### 6. Build graphs to all the edge cases

description


In [6]:
async_graph_undefined = function.set_topology("flow",engine="async")
async_graph_undefined.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
async_graph_undefined.plot()


TypeError: cannot unpack non-iterable NoneType object

#### Run using mock

descteption

In [ ]:
# 4. create mock server and test it locally (no k8s / deployment)

mock_server = function.to_mock_server()

In [ ]:
mock_server

In [ ]:
from random import choice
from datetime import datetime

iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")

time_before = datetime.now()
print("Before invoke:", time_before.strftime("%H:%M:%S.%f"))
moke_response = mock_server.test(
    "/",
    body={
        "models": None,
        "inputs": [data_point, data_point],
    },
)
time_after = datetime.now()
print("After invoke:", time_after.strftime("%H:%M:%S.%f"))
total_time = (time_after - time_before).total_seconds()



In [ ]:
print(f"Total time: {total_time:.3f} seconds")

In [ ]:
#moke_response

#### 7. Deploying Your Function

Running this cell will deploy your serving function to the cluster. This also deploys the real-time monitoring functions for your project, which are configured to track the serving function's performance and detect model drift.

In [ ]:
function.deploy()

In [ ]:
iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")


print("Before async_sync_function invoke:", datetime.now().strftime("%H:%M:%S.%f"))
deploy_response = function.invoke(
    "/",
    body={
        "models": None,
        "inputs": [data_point, data_point],
    },
)
print("After invoke:", datetime.now().strftime("%H:%M:%S.%f"))
total_time = (time_after - time_before).total_seconds()


In [ ]:
print(f"Total time: {total_time:.3f} seconds")

In [ ]:
#deploy_response